<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [6]:
!pip3 install beautifulsoup4
!pip3 install requests

In [7]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [8]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [9]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [11]:
import requests
from bs4 import BeautifulSoup

# URL de la página de Wikipedia de Falcon 9
static_url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Realizar la solicitud HTTP GET
response = requests.get(static_url)

# Verificar si la solicitud fue exitosa (código 200)
if response.status_code == 200:
    print("Solicitud HTTP exitosa")
else:
    print("Error en la solicitud HTTP:", response.status_code)



Solicitud HTTP exitosa


Create a `BeautifulSoup` object from the HTML `response`


In [12]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
response = requests.get(static_url)

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [13]:
# Use soup.title attribute
soup = BeautifulSoup(response.text, 'html.parser')

# Imprimir el título de la página para verificar
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [18]:
# Crear un objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Verificar el título de la página para asegurarnos de que se parseó correctamente
print("Título de la página:", soup.title.string)



Título de la página: List of Falcon 9 and Falcon Heavy launches - Wikipedia


Starting from the third table is our target table contains the actual launch records.


In [22]:
import requests
from bs4 import BeautifulSoup

# URL de la página de Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Enviar una solicitud HTTP a la página
response = requests.get(url)

# Crear un objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Verificar el título de la página
print("Título de la página:", soup.title.string)

# Encontrar todas las tablas en la página
tables = soup.find_all('table')

# Seleccionar la tercera tabla que contiene los registros de lanzamientos
target_table = tables[2]

# Extraer los nombres de las columnas de la tabla
column_names = []
for th in target_table.find_all('th'):
    column_names.append(th.text.strip())

# Mostrar los nombres de las columnas
print("Nombres de las columnas:", column_names)


Título de la página: List of Falcon 9 and Falcon Heavy launches - Wikipedia
Nombres de las columnas: ['Date and time (UTC)', 'Version,booster[c]', 'Launch site', 'Payload[d]', 'Orbit', 'Customer']


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [27]:
# Crear un objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Verificar el título de la página
print("Título de la página:", soup.title.string)


Título de la página: List of Falcon 9 and Falcon Heavy launches - Wikipedia


Check the extracted column names


In [28]:
# Encontrar todas las tablas en la página
tables = soup.find_all('table')

# Seleccionar la tercera tabla que contiene los registros de lanzamientos
target_table = tables[2]


In [31]:

# Obtener todos los elementos <th> del documento HTML usando BeautifulSoup
headers = target_table.find_all('th')

# Lista para almacenar nombres de columna extraídos
column_names = []

# Función para extraer nombres de columna de un <th>
def extract_column_from_header(header):
    return header.get_text(separator=" ", strip=True)

# Iterar a través de cada elemento <th> y aplicar extract_column_from_header()
for header in headers:
    column_name = extract_column_from_header(header)
    if column_name is not None and len(column_name) > 0:
        column_names.append(column_name)

# Imprimir los nombres de columna extraídos
print("Nombres de las columnas extraídas:")
print(column_names)




Nombres de las columnas extraídas:
['Date and time ( UTC )', 'Version, booster [c]', 'Launch site', 'Payload [d]', 'Orbit', 'Customer']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [33]:
import requests
import pandas as pd

# URL de la API de SpaceX
url_launches = 'https://api.spacexdata.com/v4/launches'

# Realizar la solicitud GET
response_launches = requests.get(url_launches)
data_launches = response_launches.json()

# Convertir la respuesta en un DataFrame
df_launches = pd.json_normalize(data_launches)

# Mostrar las columnas disponibles
print(df_launches.columns)


Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [35]:
# Lista de nombres de columnas extraídas en el paso anterior
column_names = [
    "Flight No.", "Date", "Time", "Version Booster", "Launch site", 
    "Payload", "Payload mass", "Orbit", "Customer", "Launch outcome", 
    "Booster landing"
]

# Crear un diccionario vacío con claves de los nombres de columnas
launch_dict = {col: [] for col in column_names}

# Función para extraer nombres de columna de un <th>
def extract_column_from_header(header):
    return header.get_text(separator=" ", strip=True)

# Iterar a través de las tablas relevantes
for table in tables:
    for row in table.find_all('tr'):
        # Verificar si la primera celda es un número de vuelo y si tiene un elemento th
        if row.th and row.th.string and row.th.string.strip().isdigit():
            cells = row.find_all('td')
            if len(cells) < 8:
                continue

            # Ejemplo de extracción de datos; ajustar según la estructura de la tabla
            flight_number = row.th.text.strip()
            date_time = cells[0].text.strip().split()
            date = date_time[0] if len(date_time) > 0 else ''
            time = date_time[1] if len(date_time) > 1 else ''
            version_booster = cells[1].text.strip()
            launch_site = cells[2].text.strip()
            payload = cells[3].text.strip()
            payload_mass = cells[4].text.strip()
            orbit = cells[5].text.strip()
            customer = cells[6].text.strip()
            launch_outcome = cells[7].text.strip()
            booster_landing = cells[8].text.strip() if len(cells) > 8 else ''

            # Añadir los datos extraídos al diccionario
            launch_dict['Flight No.'].append(flight_number)
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
            launch_dict['Version Booster'].append(version_booster)
            launch_dict['Launch site'].append(launch_site)
            launch_dict['Payload'].append(payload)
            launch_dict['Payload mass'].append(payload_mass)
            launch_dict['Orbit'].append(orbit)
            launch_dict['Customer'].append(customer)
            launch_dict['Launch outcome'].append(launch_outcome)
            launch_dict['Booster landing'].append(booster_landing)



After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [36]:
import pandas as pd

# Asumiendo que launch_dict ya está lleno con los datos del lanzamiento

# Crear un DataFrame desde el diccionario
df = pd.DataFrame(launch_dict)

# Exportar el DataFrame a CSV
df.to_csv('spacex_web_scraped.csv', index=False)

# Verificar que se haya exportado correctamente
print("DataFrame exportado a spacex_web_scraped.csv")
# Crear un DataFrame desde el diccionario
df = pd.DataFrame(launch_dict)

# Mostrar las primeras filas del DataFrame para verificar
print(df.head())



DataFrame exportado a spacex_web_scraped.csv
  Flight No. Date     Time       Version Booster   Launch site  \
0        195    3  January  F9 B5 ♺ B1060.15[22]  CCSFS,SLC-40   
1        196   10  January       F9 B5 ♺ B1076.2  CCSFS,SLC-40   
2        197   18  January       F9 B5 ♺ B1077.2  CCSFS,SLC-40   
3        198   19  January          F9 B5B1075.1   VSFB,SLC-4E   
4        199   26  January        F9 B5 ♺B1067.9  CCSFS,SLC-40   

                                             Payload  \
0   Transporter-6: (115 payloads Smallsat Rideshare)   
1  OneWeb Flight #16 / SpaceX Flight 2[28][29] (4...   
2      USA-343 / GPS III-06 (Amelia Earhart)[39][40]   
3                 Starlink Group 2–4 (51 satellites)   
4                 Starlink Group 5-2 (56 satellites)   

                 Payload mass      Orbit  Customer Launch outcome  \
0                  Unknown[e]        SSO   Various        Success   
1        6,000 kg (13,000 lb)  Polar LEO    OneWeb        Success   
2         4,35

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [37]:
# Exportar el DataFrame a CSV
df.to_csv('spacex_web_scraped.csv', index=False)

# Verificar que se haya exportado correctamente
print("DataFrame exportado a spacex_web_scraped.csv")


DataFrame exportado a spacex_web_scraped.csv


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
